## clinical variable retrieval codebase.
each querey is referencing a sql querey linked in my github for ALL patients in the database, then generating a dataframe, then paring that dataframe down to only the patients/icustay_id in our cohort. 
* 5-16-19 heavily streamlined, can now change global variables at top of page which will correspond to all variables. added all code into functions and made a composite function to run each variable. 
* each variable is also deleted to reduce rolling memory usage


In [8]:
#7-15-18
#the final version of this notebook seeks to accomplish a few tasks:
#organize and standardize all sql code so that i am running a .sql file rather than typing sql code into jupyter
#change all filepaths to match the github linked directory to ensure better version control
#extract all of the structured clinical variables we need for our project


##1/28/19: updated final version to be more generalizable and adjustable for the datewindow. 
##could be cleaned up as some variabls are time-windowed in line and some are time windowed at the end, but all is accounted for and cleaned up and optimized.

time: 1.34 ms


In [9]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 21.5 ms


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:19: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.


In [10]:
from pathlib import Path
os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


time: 1.17 ms


In [25]:
#patients of interest from rotation_cohort_generation

final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/04042019_final_pt_df2.csv', index_col=0)

patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 96 ms


## Access MIMIC database and convert it to dataframe in Pandas

In [12]:
# note, all server information is stored in a config.py file that is present in the .gitignore
import importlib
scriptName = 'config.py'
script = importlib.import_module(wd+'/notebooks/PIPELINE/%s' % scriptName)

#import config
conn = psycopg2.connect(dbname=config.dbname, user=config.user, host=config.host, port=config.port,password=config.password)
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

Password: ·····


time: 2.29 s


In [13]:
#input the sql_exe_show object and get dataframe for only patients in patient list out. 
def sql_exe_show(sql_sentence):
    cur.execute(sql_sentence)
    rows = cur.fetchall()
    col = []
    for i in range(len(cur.description)):
        col.append(cur.description[i][0])
    table = pd.DataFrame(rows,columns=col)
    return table

def sql_to_df_icu(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['icustay_id'].isin(icustay_id)]
    return sql_exe_show_df

def sql_to_df_patients(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['subject_id'].isin(patients)]
    return sql_exe_show_df

def sql_to_df_hadm(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['hadm_id'].isin(hadm_id)]
    return sql_exe_show_df

def clinvar_fxn(var_name, path, subject_id_override=False):
    f= open(path, 'r')
    var = f.read()
    cur.execute('rollback')
    cur.execute(var)
    
    if subject_id_override==True:
        df= sql_to_df_patients(sql_exe_show('select * from %s;' %(var_name)))
    
    else:
        
        try:
            df= sql_to_df_icu(sql_exe_show('select * from %s;' %(var_name)))
        except KeyError or NameError:
            try:      
                df= sql_to_df_hadm(sql_exe_show('select * from %s;' %(var_name)))
            except KeyError or NameError:
                df= sql_to_df_patients(sql_exe_show('select * from %s;' %(var_name)))

    print(df.shape)
    return(df)


time: 66.7 ms


# extracting clinical data for our patients
## IMPORTANT, USE THIS TO TUNE TIMEWINDOW OF EXTRACTION AND FOLDER TO SAVE IN

In [29]:
# lower_window=0
# upper_window=2
# time_var="t_0"
# folder="48_hr_window"
# date='16052019'

# time_col="charttime"
# time_var= 't_0'
# patient_df= final_pt_df2

# def filter_subset_save(df):
#     def time_window_filter(df, final_pt_df2,timecol,upper_window, lower_window, time_var='t_0'):
#     global lower_window, upper_window, time_var, timecol, time_var
#     vitals_all_nosummary_72 = time_window_filter(vitals_all_nosummary_df, final_pt_df2, "charttime",time_var=time_var, lower_window=lower_window, upper_window=upper_window)


date='03062019'
lower_window=0
upper_window=3
time_var="t_0"
time_col="charttime"
folder="72_hr_window_update"


time: 4.03 ms


In [15]:
###### PIPELINE BELOW #######

time: 404 µs


# rearranging pipeline so it can 
* extract vitals first, then will limit patients to those with appropriate vitals going forward. 
* save and delete after extracting to reduce unnescessary memory load

### for all variables:
* extract variable for only patients in minimum vitals list (will  be new final_pt_df)
* 

# important functions

In [21]:
def time_window_filter(df, final_pt_df2,timecol,upper_window, lower_window, time_var='t_0'):
    """
    will take in any df and filter to only values between lower_window and upper_window. 
    """
    #global upper_window, lower_window 
    
    try:
        df= pd.merge(df, final_pt_df2[['icustay_id',time_var]], left_on= 'icustay_id', right_on = 'icustay_id') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:]
    except KeyError or NameError:
        df= pd.merge(df, final_pt_df2[['hadm_id',time_var]], left_on= 'hadm_id', right_on = 'hadm_id') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:]
    return(df_after_t0)

time: 16.2 ms


## Vital Sign

In [33]:
##-- This query pivots the vital signs for the first 24 hours of a patient's stay
##-- Vital signs include heart rate, blood pressure, respiration rate, and temperature

vitals_all_nosummary_df= clinvar_fxn(
    'vitals_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/vitals_all_nosummary.sql' 
)

(14609367, 7)
time: 19min 8s


In [34]:
vitals_all_nosummary_df.head()

,subject_id,hadm_id,icustay_id,charttime,valueuom,vitalid,valuenum
3,9,150750,220597,2149-11-11 00:00:00,mmHg,MeanArtPress,81.0
4,9,150750,220597,2149-11-11 19:00:00,mmHg,MeanArtPress,72.0
5,9,150750,220597,2149-11-11 19:45:00,mmHg,MeanArtPress,76.0
6,9,150750,220597,2149-11-11 20:00:00,mmHg,MeanArtPress,99.0
17,25,129635,203487,2160-11-02 15:00:00,mmHg,MeanArtPress,82.0


time: 162 ms


# filtering to patients with bare minimum vital numbers
06.1-QC_and_missingness found that 3% or so of patients don't have baseline vitals counts. this is filtering the patients to only those who have this baseline value.

as of 10/12/18, this code has not been implemented in here and is performed after importing. (updated below to be self contained in 1/28/19)

In [36]:
#using origional criteria to find pts who have atleast 1 spo2 reading within 3 days of t_0

#The idea is that this should be the bare minimum amount of data for a patient, and without it, it's likely the physicians did not suspect an infection in these patients. 
##NOTE: this should not change when the clinical timewindow of analysis interest changes. 


vitals_filter = time_window_filter(vitals_all_nosummary_df, final_pt_df2, "charttime",time_var='t_0', lower_window=0,upper_window=3 )

vitals_filter= vitals_filter.loc[
    vitals_filter['vitalid'].notnull(),:]#.count() #6930 NULL values

icustay_id_vitals = (vitals_filter.loc[
    vitals_filter.loc[:,'vitalid']=='SpO2','icustay_id'
        ].unique())

len(icustay_id) #8731
len(icustay_id_vitals) #8629
subject_id_vitals=list(final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),'subject_id'])
hadm_id_vitals= list(final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),'hadm_id'])
icustay_id_vitals= list(final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),'icustay_id'])
del(vitals_filter)

##saving the patient database and reassigning patient set to the patient set with minimum vitals
final_pt_df2_v=final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),:]
pd.DataFrame(final_pt_df2_v).to_csv("/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2_v.csv" %date) #final cohort database n=11493 subject_id’s (7/6/18)
final_pt_df2=final_pt_df2_v.copy()


time: 29 s


## filtering, subset, and composite functions to be used for the rest of the notebook

In [37]:
def df_subset(df):
    """
    was origionally how I was ensuring all df were on the minimum vitals cohort, but now this may not be needed since i reassigned the final_pt_df2 as the minimum vitals cohort.
    """
    
    try:
        df = df.loc[df.loc[:,'icustay_id'].isin(icustay_id_vitals),:]
    except KeyError or NameError:
        try:
            df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id_vitals),:]
        except KeyError or NameError:
            df = df.loc[df.loc[:,'subject_id'].isin(subject_id_vitals),:]
    return(df)

time: 17.1 ms


In [38]:
def filter_subset_save(df, savename=None, return_df=False, save=False, time_filter_override=False):
    
    """
    composite function, performs 1: time_window_filter() and 2:df_subset() to the input dataframe. this function links them together for simplifying code needed after each sql and formatting query. 
    return_df specifies if any output is spit out.
    save specifies if the file will be saved with teh savename.
    
    fxn was created on 5/16/19 and validated against the normal pipeline.
    """
    
    global date,folder,final_pt_df2, lower_window, upper_window, time_var, timecol, time_var
    
    #
    
    if time_filter_override==False:
        time_filtered= time_window_filter(df, final_pt_df2, timecol=time_col ,time_var=time_var, lower_window=lower_window, upper_window=upper_window)
    else:
        time_filtered=df
        
    time_and_subseted= df_subset(time_filtered)
    
    if save==True:
        os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling')
        if folder != None:
             address=os.getcwd()+'/data/raw/csv/%s/'%(folder)
        else:
            address = os.getcwd()+'/data/raw/csv/'
        if not os.path.exists(address):
            print(address)
            os.makedirs(address)
            
        pd.DataFrame(time_and_subseted).to_csv(address+'/%s_%s.csv' %(date, savename))       
    else: pass

    
    if return_df==False:
        del(df, time_filtered, time_and_subseted)
    
    else:
        return(time_and_subseted)
    

time: 78.9 ms


In [39]:
filter_subset_save(vitals_all_nosummary_df, savename="vitals_all_nosummary", save=True, return_df=False)
del(vitals_all_nosummary_df)

/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window_update/


KeyboardInterrupt: 

time: 3min 51s


## testing elixhauser comobridities

In [41]:
elixhauser_nosummary_df= clinvar_fxn(
    'elixhauser_quan',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/elixhauser_quan.sql',
    subject_id_override=True
)

(21864, 33)
time: 1min 26s


### now next task: redo same as above, BUT EXCLUDE CURRENT ROW FROM CUMMAX()

In [42]:
def elix():
    elix_var=list(elixhauser_nosummary_df)[3:]
    elixhauser_nosummary_df2=elixhauser_nosummary_df.copy()

    elixhauser_nosummary_df2[elix_var]=(elixhauser_nosummary_df
                                            .sort_values('stay_num', ascending=True) #sorts values so stay_num is ascending
                                            .groupby('subject_id', as_index=False)[elix_var] #groups by subject id and filters only elixhauser variable columns
                                            .agg('cummax') #takes a cummulitive max for every row
                                       )

    #now shifting the values up by 1 so the cumulitive max doesn't consider the current values: (note: couldn't get this to work in the fxn above)
    elixhauser_nosummary_df2[elix_var]=(elixhauser_nosummary_df
                                            .sort_values('stay_num', ascending=True) #sorts values so stay_num is ascending
                                            .groupby('subject_id', as_index=False)[elix_var] #groups by subject id and filters only elixhauser variable columns
                                            .shift(fill_value=np.nan)[:-1]) #shifts the cummulitive max up by 1 so the first row is na. 

    #restricting to hadm in use
    elixhauser_nosummary_df3= elixhauser_nosummary_df2[elixhauser_nosummary_df2['hadm_id'].isin(hadm_id)]
    
    #adding icustay_id
    elixhauser_nosummary_df3=pd.merge(elixhauser_nosummary_df3, final_pt_df2[['hadm_id','icustay_id']], how="left", left_on='hadm_id',right_on='hadm_id')
    
    return(elixhauser_nosummary_df3)

time: 31.3 ms


In [43]:
elixhauser_df=elix()

time: 372 ms


In [44]:
elixhauser_df.head()

,hadm_id,subject_id,stay_num,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,...,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,icustay_id
0,100003,54610.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209281.0
1,100007,23018.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217937.0
2,100009,533.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,253656.0
3,100016,68591.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217590.0
4,100020,9973.0,5.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,282580.0


time: 67.2 ms


#### elix qc

In [45]:
filter_subset_save(elixhauser_df, savename="elixhauser_nosummary", save=False, return_df=True,time_filter_override=True)
#filter_subset_save(age_df, savename="pt_info", save=True, return_df=False, time_filter_override=True)

,hadm_id,subject_id,stay_num,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,...,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression,icustay_id
0,100003,54610.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,209281.0
1,100007,23018.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217937.0
2,100009,533.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,253656.0
3,100016,68591.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217590.0
4,100020,9973.0,5.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,282580.0
5,100033,26773.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200853.0
6,100035,48539.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,245719.0
7,100037,58947.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,270105.0
8,100037,58947.0,2.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,270105.0
9,100053,24912.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217069.0


time: 123 ms


In [150]:
#elixhauser_nosummary_df2.loc[elixhauser_nosummary_df2['subject_id']==9973.0,['subject_id','stay_num','congestive_heart_failure']]

,subject_id,stay_num,congestive_heart_failure
13,9973.0,5.0,1.0
14007,9973.0,3.0,1.0
15290,9973.0,8.0,1.0
18057,9973.0,1.0,NaN
28839,9973.0,7.0,1.0
36825,9973.0,6.0,1.0
40044,9973.0,4.0,1.0
50567,9973.0,2.0,1.0


time: 19.7 ms


In [151]:
#elixhauser_nosummary_df2.sort_values(['subject_id','stay_num'], ascending=True).head(5)

,hadm_id,subject_id,stay_num,congestive_heart_failure,cardiac_arrhythmias,valvular_disease,pulmonary_circulation,peripheral_vascular,hypertension,paralysis,...,coagulopathy,obesity,weight_loss,fluid_electrolyte,blood_loss_anemia,deficiency_anemias,alcohol_abuse,drug_abuse,psychoses,depression
31874,150750,9.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5845,109235,19.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18664,129635,25.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
61299,197661,26.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47809,176176,33.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


time: 34.8 ms


In [182]:
#restricting to hadm in use
elixhauser_nosummary_df3= elixhauser_nosummary_df2[elixhauser_nosummary_df2['hadm_id'].isin(hadm_id)]
elix_table=round(elixhauser_nosummary_df3['stay_num'].value_counts()/len(elixhauser_nosummary_df3) *100, 2)
elix_table[elix_table>1]

1.0    64.23
2.0    18.83
3.0     7.50
4.0     3.56
5.0     1.93
6.0     1.21
Name: stay_num, dtype: float64

time: 254 ms


## importing diagnosis description upon admission written by nurses

In [48]:
admin_dx= clinvar_fxn(
    'admission_diagnosis',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/admission_diagnosis.sql',
    subject_id_override=False
)

(14668, 2)
time: 911 ms


In [52]:
string1=""
diagnosis_strings=admin_dx['diagnosis'].to_list()

time: 4.01 ms


In [60]:
string1=""
string1.join(diagnosis_strings)

TypeError: sequence item 7012: expected str instance, NoneType found

time: 11.5 ms


In [61]:
diagnosis_strings

['UPPER GI BLEED',
 'BOWEL OBSTRUCTION',
 'CORONARY ARTERY DISEASE',
 'PNEUMONIA',
 'HYPONATREMIA',
 'MEDIASTINAL AIR',
 'POST ARREST',
 'WEAKNESS',
 'HYPOTENSION',
 'CONGESTIVE HEART FAILURE;COPD',
 'INFECTED ABDOMINAL WALL;HERNIA',
 'HEART BLOCK; UTI',
 'CORONARY ARTERY DISEASE\\CARDIAC CATH',
 'CORONARY ARTERY DISEASE\\RIGHT AND LEFT HEART CATH',
 'PERFORATED BOWEL',
 'PANCREATITIS',
 'CHOLECYSTITIS',
 'PNEUMONIA;TELEMETRY',
 'DYSPNEA',
 'CONGESTIVE HEART FAILURE',
 'CARDIAC ARREST',
 'SCROTAL ABSCESS',
 'FEVER',
 'PERITONITIS',
 'PNEUMONIA',
 'CHEST PAIN\\CARDIAC CATH',
 'DIABETIC KETOACIDOSIS',
 'WEAKNESS,FAILURE TO THRIVE',
 'INTRACRANIAL HEMORRHAGE',
 'UPPER GASTROINTESTINAL BLEED',
 'STEMI',
 'HYPOTENSION,RASH,FEVER,SEIZURE',
 'S/P FALL;RESPIRATORY FAILURE',
 'RESPIRATORY DISTRESS',
 'HYPOTENSION;ALTERED MENTAL STATUS',
 'HYPOTENSION;RULE OUT SEPSIS',
 'CONGESTIVE HEART FAILURE',
 'HEPATIC FAILURE;ACUTE ABDOMEN;?SEPTIC',
 'RENAL FAILURE;PNEUMONIA',
 'PLEURAL EFFUSION',
 'GASTRO

time: 9.91 ms


In [87]:
def word_extraction(sentence):    
    ignore = ['a', "the", "is"]
    words = re.sub("[^\w]", " ",  sentence).split()
    cleaned_text = [w.lower() for w in words if w not in ignore]
    return cleaned_text


def tokenize(sentences):    
# #     words = []    
# #     for sentence in sentences:        
# #         w = word_extraction(str(sentence))
# #         words.extend(w)      
#     words = []  
#     for element in sentences:      
#         w = word_extraction(str(element))        
#         words.extend(w)  
#         words = sorted(list(set(words)))    
#         return words

    words = []  
    for element in diagnosis_strings:      
        w = word_extraction(str(element))        
        words.extend(w)   
    return(words)

time: 14.2 ms


In [90]:
def generate_bow(allsentences):        
    vocab = tokenize(allsentences)    
    print("Word List for Document \n{0} \n".format(vocab));
    for sentence in allsentences:        
        words = word_extraction(sentence)        
        bag_vector = np.zeros(len(vocab))        
        for w in words:            
            for i,word in enumerate(vocab):                
                if word == w:                     
                    bag_vector[i] += 1                            
                    print("{0}\n{1}\n".format(sentence,numpy.array(bag_vector)))


time: 5.91 ms


In [93]:
diagnosis_words=tokenize(diagnosis_strings)

time: 56.5 ms


In [ ]:

cancer_words=[
    "cancer",
    "metastases",
    "remission",
    "relapse",
    "onc",
    "oncology",
    "lymphoma",
    "leukemia",
    "tumor",
    "oma",
    "carcinoma",
    ""
]

#making all lowercase
diagnosis_words=[str(x).lower() for x in diagnosis_words]



In [ ]:
#function to parse out ABrx_Drug_list
#uses regular esxpressions and rule based parsing

d_items_df=pd.read_sql_query(d_items_sql,conn)
d_item_microbio_list=d_items_df['label'].unique().tolist()
d_item_microbio_list=[str(x).lower() for x in d_item_microbio_list]

i=0
replace_list= [
    'sodium','hcl','potassium','sulfate','potassium','monohydrate', 
    'oral susp.','ophthalmic oint', 'desensitization', r'% solution',
    'hippurate','suspension','ointment','phosphate', 'soln', 'solution',
    'ophth.','ophth.','*nf*']
    
@memory.cache
def parsingfxn(element):
    element = r'%s'%(element)
    element = re.sub(r'\b\w{1,3}\b', r"", element)
    element= element.strip(r" ")
    for j in replace_list:
        element= element.replace(j, r'')   
    element= element.strip(r" ")
    if r"**" == str(element)[0:2]:
        element = str(element)[2:]
    element = element.split(r".%")[0]
    element = element.split(r". %")[0]
    element = element.split(r"ophth*")[0]
    element = element.split(r"-")[0]
    element = element.split(r"(")[0]
    element = element.split(' ',1)[0]
    return(element)    

ABrx_Drug_list3 =[parsingfxn(x) for x in ABrx_Drug_list]
ABrx_Drug_list3= list(set(ABrx_Drug_list3))
ABrx_Drug_list3.remove('')

In [112]:
# #counting all word occurances
# from collections import Counter

# counter=Counter(diagnosis_words)

# kk=[list(counter.keys()),[int(x) for x in counter.values()]]

# df=pd.DataFrame(np.array(kk).T, columns=['Word','Count'])
# df['Count']=df['Count'].astype(int)
# df.sort_values('Count',ascending=False).head(100)

,Word,Count
8,pneumonia,1460
18,failure,1323
62,sepsis,1205
2,bleed,829
5,coronary,690
87,sda,663
6,artery,641
7,disease,629
41,pain,619
17,heart,608


time: 25.8 ms


In [104]:
[int(x) for x in counter.values()]

[305,
 371,
 829,
 139,
 161,
 690,
 641,
 629,
 1460,
 67,
 7,
 2,
 75,
 208,
 63,
 432,
 485,
 608,
 1323,
 245,
 29,
 369,
 4,
 34,
 58,
 49,
 239,
 282,
 154,
 58,
 207,
 33,
 183,
 71,
 506,
 152,
 7,
 67,
 504,
 18,
 288,
 619,
 162,
 156,
 44,
 29,
 175,
 343,
 333,
 30,
 10,
 140,
 387,
 366,
 158,
 415,
 153,
 450,
 522,
 585,
 54,
 52,
 1205,
 42,
 481,
 7,
 51,
 414,
 72,
 115,
 2,
 4,
 149,
 93,
 94,
 71,
 136,
 70,
 3,
 4,
 119,
 28,
 169,
 284,
 276,
 67,
 13,
 663,
 63,
 3,
 10,
 107,
 196,
 261,
 302,
 240,
 87,
 34,
 1,
 15,
 32,
 3,
 1,
 329,
 15,
 12,
 53,
 498,
 107,
 63,
 1,
 40,
 63,
 59,
 235,
 60,
 144,
 8,
 154,
 40,
 73,
 129,
 128,
 4,
 2,
 49,
 1,
 86,
 80,
 39,
 14,
 28,
 171,
 25,
 3,
 7,
 38,
 175,
 85,
 120,
 85,
 4,
 94,
 36,
 60,
 44,
 17,
 15,
 3,
 41,
 1,
 8,
 5,
 2,
 2,
 57,
 53,
 35,
 17,
 20,
 169,
 71,
 12,
 8,
 250,
 244,
 30,
 163,
 3,
 4,
 21,
 1,
 29,
 1,
 35,
 36,
 4,
 1,
 24,
 9,
 142,
 19,
 28,
 19,
 119,
 15,
 12,
 13,
 13,
 1,
 4,
 80,
 

time: 13.8 ms


## testing elixhauser_quan end

# demographics

In [9]:
#gender & race

pt_info_sql = query_schema + """
SELECT SUBJECT_ID, INSURANCE, LANGUAGE, RELIGION, MARITAL_STATUS, ETHNICITY
from mimiciii.admissions
ORDER BY subject_id DESC
"""

pt_info_df=pd.read_sql_query(pt_info_sql,conn) #361711 patients with sterile culture -> 374643 with addn of bal and broncho... 7/16/18


time: 639 ms


In [83]:
def demographics():
    """
    wrapping demographics code into a fxn. basically combines ethinicity, age, gender and race into one df. 
    note: age is read from a csv, and i need to look back at where the csv comes from (i believe it's from cohort selection in pipeline). 
    """
    global final_pt_df2
    pt_info_sql = query_schema + """
    SELECT SUBJECT_ID, INSURANCE, LANGUAGE, RELIGION, MARITAL_STATUS, ETHNICITY
    from mimiciii.admissions
    ORDER BY subject_id DESC
    """

    pt_info_df=pd.read_sql_query(pt_info_sql,conn) #361711 patients with sterile culture -> 374643 with addn of bal and broncho... 7/16/18
    
    ethnicity_df=(pt_info_df.loc[
    pt_info_df.loc[:,"subject_id"].isin(
        final_pt_df2['subject_id'].tolist()),:]).drop_duplicates(['subject_id','ethnicity'])
    ethnicity_df= ethnicity_df[['subject_id','ethnicity']].sort_values('ethnicity', ascending=False).groupby('subject_id', as_index=False).first()
    
    #gender
    pt_info_sql = query_schema + """
    SELECT SUBJECT_ID, GENDER
    from mimiciii.patients
    ORDER BY subject_id DESC
    """
    #admissions
    # 

    pt_info_df=pd.read_sql_query(pt_info_sql,conn) #361711 patients with sterile culture -> 374643 with addn of bal and broncho... 7/16/18
    
    #combining gender, race
    pt_info_df=(pt_info_df.loc[
        pt_info_df.loc[:,"subject_id"].isin(
            final_pt_df2['subject_id'].tolist()),:]).drop_duplicates(['subject_id','gender'])
    pt_info_df= pd.merge(ethnicity_df, pt_info_df)
    
    #age- read from csv. 
    age_df= pd.read_csv(Path(wd+'/data/processed/22112018_pt_age.csv'))
    age_df=(age_df.loc[
        age_df.loc[:,"hadm_id"].isin(
            hadm_id),:])

    #combining age, gender and race. 
    age_df=pd.merge(age_df, final_pt_df2[['hadm_id','icustay_id','t_0']])
    age_df=pd.merge(age_df[['subject_id','icustay_id','first_admit_age','t_0']],pt_info_df)

    age_df[age_df['first_admit_age']>89]['first_admit_age']=90
    
    age_df=pd.melt(age_df, id_vars=['icustay_id','subject_id','t_0'])
    age_df=age_df.rename(index=str, columns={'variable':'label'})

    age_df['delta']=pd.to_timedelta('0 days')
    age_df['uom']="N/A"
    age_df.loc[age_df.loc[:,'label']=='first_admit_age','uom']='years'

    age_df= age_df.loc[age_df.loc[:,"icustay_id"].isin(icustay_id),:]

    ###using regular expressions to reduce the # of ethinicities
    age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?BLACK')),'value']="black"
    age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?HISPANIC|PORTUGUESE')),'value']="hispanic"
    age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?WHITE')),'value']="white/nonhispanic"
    age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'.*?ASIAN')),'value']='asian'
    age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'(UNKNOWN|MULTI|UNABLE|DECLINE|OTHER)')),'value']='unknown/other'
    age_df.loc[(age_df.loc[:,"label"]=='ethnicity') & (age_df.loc[:,"value"].str.contains(r'[AZ]+')),'value']="unknown/other" #lumping all other low n values into other

    #age_df.loc[age_df.loc[:,"label"]=='ethnicity','value'].value_counts()
    return(age_df)



time: 118 ms


In [84]:
age_df= demographics()

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


time: 51 s


/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:59: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.


In [90]:
filter_subset_save(age_df, savename="pt_info", save=True, return_df=False, time_filter_override=True)
del(age_df)

time: 777 ms


## Echodata in Noteevents- sense removed

In [21]:
# echodata_df= clinvar_fxn(
#     'echodata',
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/echodata.sql' 
# )

time: 1.12 ms


## Weight

In [92]:
# -- This query extracts weights for adult ICU patients on their first ICU day.
# -- It does *not* use any information after the first ICU day, as weight is
# -- sometimes used to monitor fluid balance.

weightfirstday_df= clinvar_fxn(
    'weightfirstday',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/weightfirstday.sql' 
)
weightfirstday_df['uom']='kg'

(14668, 6)
time: 2.71 s


In [95]:
filter_subset_save(weightfirstday_df, savename="weightfirstday", save=True, return_df=False, time_filter_override=True)
del(weightfirstday_df)

time: 94.3 ms


## Height

In [97]:
# -- This query extracts heights for adult ICU patients.
# -- It uses all information from the patient's first ICU day.
# -- This is done for consistency with other queries - it's not necessarily needed.
# -- Height is unlikely to change throughout a patient's stay.
heightfirstday_df= clinvar_fxn(
    'heightfirstday',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/heightfirstday.sql' 
)

heightfirstday_df['uom']='cm'

(14668, 4)
time: 1.35 s


In [98]:
filter_subset_save(heightfirstday_df, savename="heightfirstday", save=True, return_df=False, time_filter_override=True)
del(heightfirstday_df)

time: 83.1 ms


# Labs

In [99]:
# -- This query pivots lab values for all patients, then filtered to those in my cohort.
# -- Have already confirmed that the unit of measurement is always the same: null or the correct unit

labs_all_nosummary_df= clinvar_fxn(
    'labs_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/labs_all_nosummary.sql' 
)

(3267301, 6)
time: 3min 4s


In [100]:
#importing unit of mesurements:
def uom_sql_import(file_path):
    if isinstance(file_path, str)== True:
        f = open(Path(file_path), 'r')
    else:
        f = open(Path(str(file_path)), 'r')

    SQL = open(file_path,'r').read()
    SQL_df= pd.read_sql_query(SQL,conn)   
    return(SQL_df)

time: 35.6 ms


In [101]:
lab_uom= uom_sql_import(Path(wd+'/data/raw/clinical_var_sql/unit_of_mesurement/labs_uom.sql'))
labs_all_nosummary_df = pd.merge(labs_all_nosummary_df, lab_uom, left_on='label', right_on='label')

labs_all_nosummary_df[labs_all_nosummary_df['label']=='LYMPHO%']#.value_counts() #4-15-19: what is this, is this exploring absolute lymphocyte %?
#labs_all_nosummary_df['label'].value_counts()

labs_all_nosummary_df[labs_all_nosummary_df['label']=='WBC']['valuenum'].describe()
#labs_all_nosummary_df[labs_all_nosummary_df['label']=='PLATELET']['valuenum'].describe()


count    192280.000000
mean         11.847600
std           8.971701
min           0.090000
25%           7.300000
50%          10.300000
75%          14.400000
max         471.700000
Name: valuenum, dtype: float64

time: 25 s


In [102]:
filter_subset_save(labs_all_nosummary_df, savename="labs_all_nosummary", save=True, return_df=False, time_filter_override=False)
del(labs_all_nosummary_df)
del(lab_uom)

NameError: name 'labs_all_nosummary' is not defined

time: 19.1 s


## Glasgow Coma Scale

In [105]:
##depreciated 08/28/18
# gcsall_df= clinvar_fxn(
#     'gcsall',
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/gcsall.sql' 
# )

(14668, 8)
time: 56.8 s


In [123]:
#modified on 8/28/18 to have the days annotation.
##--8/28/18: added in epoch as days, in order to help determine btwn t_0 and 72 hour for pts.
gcsall_days_df= clinvar_fxn(
    'gcsall_days',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/gcsall_days.sql' 
)

(14668, 9)
time: 54.2 s


In [124]:
#adding in icu_admit time and filtereing time_var to time window.
def gcs_72(gcsall_days_df,final_pt_df2, time_var='t_0', lower_window=0, upper_window=3):
    """
    written a while back, aims to adding in icu_admit time and filtereing time_var to time window. will use this with time_filter_override=True in my filter_subset_save()
    """
    
    
    ##merging gcsall_days_df with final_pt df in order to append on icustay_id, the time var, and ICU_admit
    gcsall_days_df_merge= pd.merge(
    gcsall_days_df,
    final_pt_df2[['icustay_id','ICU_admit',time_var]],
    left_on='icustay_id',
    right_on='icustay_id')

    gcsall_days_df_merge['day'] = gcsall_days_df_merge['day']-1 #putting the epoch days so that 0 = the first day
    
    #approximating the charttime of the time associated with each gcs score
    gcsall_days_df_merge['approx_charttime']=pd.to_timedelta((gcsall_days_df_merge['day'])*24, unit='h') + pd.to_datetime(gcsall_days_df_merge['ICU_admit'])
    
    # day # + ICU_admission day.
    gcsall_days_df_merge['admit_plus_day']= (
        pd.to_datetime(gcsall_days_df_merge['ICU_admit'])
        + pd.to_timedelta(gcsall_days_df_merge['day'], unit='D')
    )
    
    #difference between the admission+epoch day - time_var.
    gcsall_days_df_merge['delta']= (
        pd.to_datetime(gcsall_days_df_merge['admit_plus_day']) - pd.to_datetime(gcsall_days_df_merge[time_var])
    )
    
    #filtering day windows
    gcsall_days_df_merge_72= (
        gcsall_days_df_merge.loc[gcsall_days_df_merge.loc[:,'delta']>= pd.Timedelta(days=lower_window),:])
    gcsall_days_df_merge_72= (
        gcsall_days_df_merge_72.loc[gcsall_days_df_merge_72.loc[:,'delta']<= pd.Timedelta(days=upper_window),:])
    return(gcsall_days_df_merge_72)

time: 23.6 ms


In [125]:
gcs72_df = gcs_72(gcsall_days_df,final_pt_df2, time_var=time_var, lower_window=lower_window,upper_window=upper_window )
gcs72_df['uom']='GCS_score' #adding in uom


time: 109 ms


In [126]:
filter_subset_save(gcs72_df, savename="gcs", save=True, return_df=False, time_filter_override=True)
del(gcs72_df)

time: 171 ms


## Renal replacement therapy

the sql code for this was not equipped to join all of the charttimes together. so i decided to do it in python below.
the rrt_all_df code above was only a 1 or 0 if patient had RRT during their entire icu stay. 

- step 1: run all sql codes
- 2: filter on only the t_0 to t_72 hour rows
- 3: filter on the 1223 patients who have a positive value
- 4: get the earliest incidence of rrt for each 1223 patients. 

In [118]:
def rrt_runmerge():
    """
    wrapping a lot of scripting into a function. grabs the 5 different rrt datas, filters them to timewindow, and merges them into 1 dataframe.
    """
    global date,folder, patient_df,lower_window, upper_window, time_var, time_var, time_col
    
    ###5 sql queries to grab raw data
    
    #mv_ce
    f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_mv_ce.sql', 'r')
    rrtSQL_mv_ce = f.read()
    rrtSQL_mv_ce_sql = query_schema + rrtSQL_mv_ce.format(tuple(patients))
    rrtSQL_mv_ce_df=pd.read_sql_query(rrtSQL_mv_ce_sql,conn)    
    #cv
    f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_cv.sql', 'r')
    rrtSQL_cv = f.read()
    rrtSQL_cv_sql = query_schema + rrtSQL_cv.format(tuple(patients))
    rrtSQL_cv_df=pd.read_sql_query(rrtSQL_cv_sql,conn)          
    #mv_ie
    f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_mv_ie.sql', 'r')
    rrtSQL_mv_ie = f.read()
    rrtSQL_mv_ie_sql = query_schema + rrtSQL_mv_ie.format(tuple(patients))
    rrtSQL_mv_ie_df=pd.read_sql_query(rrtSQL_mv_ie_sql,conn)      
    rrtSQL_mv_ie_df['charttime']= rrtSQL_mv_ie_df['starttime']
    rrtSQL_mv_ie_df=rrtSQL_mv_ie_df.drop('starttime', axis=1)
    #mv_de
    f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_mv_de.sql', 'r')
    rrtSQL_mv_de = f.read()
    rrtSQL_mv_de_sql = query_schema + rrtSQL_mv_de.format(tuple(patients))
    rrtSQL_mv_de_df=pd.read_sql_query(rrtSQL_mv_de_sql,conn)      
    #mv_pe
    f = open('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/rtt_mv_pe.sql', 'r')
    rrtSQL_mv_pe = f.read()
    rrtSQL_mv_pe_sql = query_schema + rrtSQL_mv_pe.format(tuple(patients))
    rrtSQL_mv_pe_df=pd.read_sql_query(rrtSQL_mv_pe_sql,conn)          
    rrtSQL_mv_pe_df['charttime']= rrtSQL_mv_pe_df['starttime']
    rrtSQL_mv_pe_df=rrtSQL_mv_pe_df.drop('starttime', axis=1)
    
    ### timewindow filtering

    def hour_72_window_rrt(df, final_pt_df2,timecol='charttime',time_var='t_0', lower_window=0, upper_window=3 ):
        ##modified to make more generalizable to easily accomidate PA cohort but default to my origional cohort.
        ##filters rrt to within timewindow  between timecol- time_var

        df= pd.merge(final_pt_df2[['icustay_id',time_var]], df, left_on= 'icustay_id', right_on = 'icustay_id', how='left') #n=240317
        df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
        df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
        df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:] 
        #df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'rrt']==1,:].groupby('icustay_id')['charttime'].min()
        return(pd.DataFrame(df_after_t0))#.reset_index())
    
    rrtSQL_mv_ce_pt =hour_72_window_rrt(rrtSQL_mv_ce_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    rrtSQL_cv_pt =hour_72_window_rrt(rrtSQL_cv_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    rrtSQL_mv_ie_pt =hour_72_window_rrt(rrtSQL_mv_ie_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    rrtSQL_mv_de_pt =hour_72_window_rrt(rrtSQL_mv_de_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    rrtSQL_mv_pe_pt =hour_72_window_rrt(rrtSQL_mv_pe_df, patient_df, timecol=time_col,time_var=time_var, lower_window=lower_window,upper_window=upper_window)
    
    ### merging all 5 filtered rrt_df together
    
    def rrt_merging(rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt, timecol='charttime',time_var='t_0'):
        ###returns an aggregate y/n of if patient had positive rrt within timewindow. 

        rrt_merged_pt= pd.concat([rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt])

        #making a 1 if has positive rrt within timewindow:
        rrt_merged_pt= pd.DataFrame(rrt_merged_pt.loc[rrt_merged_pt.loc[:,'rrt']==1,:].groupby('icustay_id')[timecol].min().reset_index())
        rrt_merged_pt['rrt']=1

        rrt_merged_allpt_df= pd.merge(final_pt_df2[['icustay_id',time_var]], rrt_merged_pt, left_on= 'icustay_id', right_on = 'icustay_id', how='left') #n=240317
        rrt_merged_allpt_df=rrt_merged_allpt_df.rename(index=str, columns={timecol:"first_charttime"})

        rrt_merged_allpt_df['uom']='category' #adding a uom category
        rrt_merged_allpt_df.loc[rrt_merged_allpt_df.loc[:,'rrt'].isnull(),'rrt']='0'

        return(rrt_merged_allpt_df)


    rrt_merged_allpt_df= rrt_merging(rrtSQL_mv_ce_pt, rrtSQL_cv_pt, rrtSQL_mv_ie_pt, rrtSQL_mv_de_pt, rrtSQL_mv_pe_pt, timecol=time_col,time_var=time_var)
    return(rrt_merged_allpt_df)

time: 196 ms


In [120]:
rrt_merged_allpt_df= rrt_runmerge()
filter_subset_save(rrt_merged_allpt_df, savename="rrt_merged", save=True, return_df=False, time_filter_override=True)
del(rrt_merged_allpt_df)

time: 51.3 ms


## Urine Output

In [ ]:
##depreciated
# urine_output_all_df= clinvar_fxn(
#     'urine_output_all',
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/urine_output_all.sql' 
# )

# UTI related variables

In [127]:
uti_all_df= clinvar_fxn(
    'uti_all',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/uti_all.sql' 
)

(708156, 12)
time: 57.1 s


In [128]:
filter_subset_save(uti_all_df, savename="uti", save=True, return_df=False, time_filter_override=False)
del(uti_all_df)

time: 6.35 s


## Blood Gas Test

In [129]:
def PaO2(bg_all_nosummary_df):
    """
    overview: replaces the PO2 label with PaO2 on all instances (defined as sharing icustay_id and charttime being equal)
        where the specimen label == 'ART'
        
    input: bloodgas dataframe with values annotated. 
    output: bloodgas dataframe with values annotated where PO2 label is replaced with PaO2 according to above criteria
    """
    
    #making a unique varaible to search for and mark all rows where time and icustay_id has an art flag
    bg_all_nosummary_df['unique_var']= bg_all_nosummary_df['icustay_id'].map(str) +  bg_all_nosummary_df['charttime'].map(str)

    #making subset dataframe for label == SPECIMEN 
    bg_all_nosummary_specimen= bg_all_nosummary_df.loc[bg_all_nosummary_df.loc[:,'label']=='SPECIMEN',:]
    
    #all ART related rows: unique_var for all rows where label== SPECIMEN
    bg_all_nosummary_ART = bg_all_nosummary_specimen[bg_all_nosummary_specimen['value']=='ART'] 
    bg_all_nosummary_ART_list= list(bg_all_nosummary_ART['unique_var'].unique())

    #two criteria needed to change the PO2 to PaO2 label.
    criteria1=(bg_all_nosummary_df['label'] == 'PO2')
    criteria2=(bg_all_nosummary_df['unique_var'].isin(bg_all_nosummary_ART_list))
    
    #making changes
    bg_all_nosummary_df.loc[(criteria2 & criteria1),'label']= 'PaO2'
    
    return(bg_all_nosummary_df)

time: 20.1 ms


In [130]:
bg_all_nosummary_df= clinvar_fxn(
    'bg_all_nosummary',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/no_summary/bg_all_nosummary.sql' 
) 
bg_all_nosummary_df = PaO2(bg_all_nosummary_df)


(1664578, 8)
time: 1min 34s


In [131]:
filter_subset_save(bg_all_nosummary_df, savename="bg_nosummary", save=True, return_df=False, time_filter_override=False)
del(bg_all_nosummary_df)

time: 13.9 s


In [ ]:
#bg_all_nosummary_df.head()

## Vaso_active therapies

In [132]:
# 10/12/18 added amountuom as amount_uom, rateuom as rate_uom to many lines of the sql code.

weightdurations_df= clinvar_fxn(
    'weightdurations',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/weightdurations.sql'
)
#
epi_dose_df= clinvar_fxn(
    'epinephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/epinephrine_dose.sql'
)
#
norepi_dose_df= clinvar_fxn(
    'norepinephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/norepinephrine_dose.sql'
)
#
dopamine_dose_df= clinvar_fxn(
    'dopamine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/dopamine_dose.sql'
)
#
dobutamine_dose_df= clinvar_fxn(
    'dobutamine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/dobutamine_dose.sql'
)
#
vasopressin_dose_df= clinvar_fxn(
    'vasopressin_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/vasopressin_dose.sql'
)

#removing units/hour, as these are not appropriate
vasopressin_dose_df= vasopressin_dose_df.loc[~vasopressin_dose_df.loc[:,'rate_uom'].isin(['Uhr','units/hour']),:]
#
phenylephrine_dose_df= clinvar_fxn(
    'phenylephrine_dose',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/phenylephrine_dose.sql'
)

(46991, 4)
(3637, 7)
(44522, 7)
(9941, 7)
(3199, 7)
(9280, 7)
(31665, 7)
time: 1min 16s


In [133]:
weightdurations_df= clinvar_fxn(
    'weightdurations',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/weightdurations.sql'
)

(46991, 4)
time: 4.26 s


In [134]:
#adding an identification label column and merging them into 1 df. 
epi_dose_df['label']='epinephrine'
norepi_dose_df['label']='norepinephrine'
dopamine_dose_df['label']='dopamine'
dobutamine_dose_df['label']='dobutamine'
vasopressin_dose_df['label']='vasopressin'
phenylephrine_dose_df['label']='phenylephrine'
vaso_dose_df = pd.concat([epi_dose_df, norepi_dose_df, dopamine_dose_df, dobutamine_dose_df, vasopressin_dose_df,phenylephrine_dose_df ])


time: 302 ms


In [136]:
filter_subset_save(vaso_dose_df, savename="vaso_dose", save=True, return_df=False, time_filter_override=True)
del(vaso_dose_df)
del(epi_dose_df, norepi_dose_df, dopamine_dose_df, dobutamine_dose_df, vasopressin_dose_df,phenylephrine_dose_df)

time: 888 ms


# ventilator settings and categorization

In [142]:
#ventsettings_df = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/15082018_ventsettings_df.csv', index_col=0)
ventsettings_df= clinvar_fxn(
    'ventsettings',
    '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/clinical_var_sql/all/ventsettings.sql'
)
#going from wide format to long:
#pd.melt(ventsettings_df, id_vars=['icustay_id','charttime'])

(710323, 6)
time: 2min 36s


In [141]:
# filter_subset_save(ventsettings_df, savename="ventsettings", save=True, return_df=False, time_filter_override=False)
# del(ventsettings_df)

time: 5.87 s


In [143]:
def vent_data(vent_df,time_var='t_0', lower_window=0, upper_window=3 ):
    df= pd.merge(vent_df,
                final_pt_df2[['icustay_id',time_var]],
                left_on='icustay_id',
                right_on= 'icustay_id',
                how='left')
    df['delta']= pd.to_datetime(df['charttime']) - pd.to_datetime(df[time_var])
    df_timewindow= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
    df_timewindow= df_timewindow.loc[df_timewindow.loc[:,'delta']<= pd.Timedelta(days=upper_window),:] 
    df_timewindow['day']= df_timewindow['delta'].apply(lambda x: pd.to_timedelta(x,unit='d').days) #day #
      
    return(df_timewindow)

#df_timewindow =vent_data(ventsettings_df,time_var='first_pos_else_neg_ssc', lower_window=-1, upper_window=1 )

def vent_day_categorizer(vent_df,time_var='t_0', lower_window=0, upper_window=3 ):
    
    df_timewindow =vent_data(vent_df,time_var=time_var, lower_window=lower_window, upper_window=upper_window)
    
    df_timewindow_perday=df_timewindow.groupby(['icustay_id','day'], as_index=False)[['mechvent','oxygentherapy']].agg({'mechvent':'max', 'oxygentherapy':'max'})

    conditions= [
        (df_timewindow_perday['mechvent']==1),
        ((df_timewindow_perday['oxygentherapy']==1) & (df_timewindow_perday['mechvent']==0)),
        (df_timewindow_perday['oxygentherapy']==0 & (df_timewindow_perday['mechvent']==0))]

    choices=['Mech', 'Oxygen', 'None']
    #
    df_timewindow_perday['value']= np.select(conditions, choices, default='error:no_value_filled')
    df_timewindow_perday['value']
    df_timewindow_perday=df_timewindow_perday.reset_index()
    df_timewindow_perday['uom']= 'mech/O2/none category'
    df_timewindow_perday= df_timewindow_perday.drop(['mechvent','oxygentherapy','index'], axis=1)
    df_timewindow_perday=pd.merge(df_timewindow_perday, final_pt_df2[['icustay_id',time_var]] )
    return(df_timewindow_perday)


time: 49.8 ms


In [144]:
#ventcategory_df = vent_categorization(final_pt_df2, ventsettings_df, time_var='first_pos_else_neg_ssc' )
ventcategory_df= vent_day_categorizer(ventsettings_df,time_var=time_var, lower_window=lower_window, upper_window=upper_window)
#ventcount_df = vent_count(final_pt_df2,ventsettings_df, time_var='first_pos_else_neg_ssc')

time: 3.52 s


In [146]:
filter_subset_save(ventcategory_df, savename="ventcategory", save=True, return_df=False, time_filter_override=True)
#del(ventcategory_df)

time: 331 ms


In [ ]:
ventcategory_df.head()

# daily SOFA score

### running yiheng's sql codes to capture daily sofa_score. ideally i coulda just used my data above but she had this written already so i'll use this.

link to her github: https://github.com/yihengpan/fluid_management/tree/master/sofa

###requirements for sofa_pan
sofa <- scorecalc <- scorecomp <- vaso_cv   
                                 <-wt      
                                 <-echo2

                   <- vaso_mv    
                   <- pafi2      <-bloodgas_pan_aterial <- bloodgas_pan  
                                 <-ventelurations
                   <- vitals_pan 
                   <- labs_pan   
                   <- uo_pan     
                   <- gcs_pan    
                                
                                

In [147]:
sofa_path= '/Users/geickelb1/Documents/GitHub/fluid_management/sofa'
var='wt'
wt_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='echo2'
echo2_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)


var='bloodgas_pan'
bloodgas_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='bloodgas_pan_arterial'
bloodgas_pan_art_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

(22488, 3)
(14668, 2)
(188079, 33)
(152414, 39)
time: 3min 11s


In [148]:
#vaso_mv, vaso_cv,pafi2, vitals_pan, labs_pan, uo_pan, gcs_pan
sofa_path= '/Users/geickelb1/Documents/GitHub/fluid_management/sofa'
var='vaso_mv'
vaso_mv_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='vaso_mv'
vaso_mv_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='vaso_cv'
vaso_cv_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='pafi1'
pafi1_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='pafi2'
pafi2_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)


var='labs_pan'
labs_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='uo_pan'
uo_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='gcs_pan'
gcs_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

###

var='scorecomp'
scorecomp_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='scorecalc'
scorecalc_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

var='sofa_pan'
sofa_pan_df= clinvar_fxn(
    var,
    Path(sofa_path+'/%s.sql' %(var))
)

(5732, 6)
(5732, 6)
(6883, 6)
(152414, 5)
(48461, 4)
(76926, 42)
(75721, 5)
(14486, 9)
(87899, 14)
(87899, 20)
(83629, 23)
time: 7min 58s


In [149]:
#deleting these to clear up memory 

del(vaso_cv_df, vaso_mv_df, labs_pan_df, gcs_pan_df, scorecalc_df, scorecomp_df, uo_pan_df, pafi1_df, pafi2_df, bloodgas_pan_art_df, echo2_df, wt_df)

time: 44.3 ms


In [150]:
sofa_pan_df['hadm_id'].nunique() #8707
final_pt_df2['hadm_id'].nunique() #8731

14181

time: 39.1 ms


In [208]:
#adding in t_0 & icuadmit date
def sofa_day_window_filter(sofa_pan_df, time_var= 't_0', lower_window= 0, upper_window=3): #'t_0'):
    import datetime
    '''
    #Yihangpan wrote a sql script and materialized view "sofa_pan" which gives the sofa score for each day in icu for each patient. 
    #since the sofa_pan has days after admission but not chartdates, I need to use day # to find the associated t_0 to t_0+72 for each patient. 
    # the challenge was that I had to relate day# in sofa_pan to my t_0 date. the day # was based on the days after icu admission, where day1 = the first day (0 to 24 hours post admission). this was changed so day 0= 0 to 24 hours.
    #To do this, I added day# (where day 0 is the first day) to icu admission date. 
    #I then filtered on only the rows where this icuadmin + day# was between t_0 and t_0 + 72 hours. 

    #since t_0 has only day resolution, and for that I ignored hours and only took the date (rounded down all hours/minutes/seconds). this is similar to how i made the t_0 date. 
    #the problem this creates is that it widens the potential time window, so it theoretically can contain up to 95.99 hours, since hours on day 1 were collapsed to 0. 

    
    input: 
        sofa_pan_df: daily sofa scores captured from sofa_pan_sql.
        optional:
            time_var: the time variable we want to base the window off of
            window_bottom= 0, time_var- window_bottom (days + time_var) = first daily sofa score to capture
            window_top= 0, time_var- window_top (days + time_var) = last daily sofa score to capture 
    
    output: sofa_pan_sql annotated with days and filtered to time window set by window_bottom and window_top. 
    '''
    #reducing to minimum vital patients
    sofa_pan_df=sofa_pan_df.loc[sofa_pan_df.loc[:,"icustay_id"].isin(icustay_id_vitals),:]

    ##merging sofa_pan with final_pt df in order to append on icustay_id, the time var, and ICU_admit
    
    sofa_df_merged= pd.merge(sofa_pan_df,
                             final_pt_df2[['icustay_id',time_var,'ICU_admit']],
                             left_on= 'icustay_id',
                             right_on = 'icustay_id',
                             how='left') #n=240317

    #sofa_df_merged['admit_t0_rounded'] = pd.to_datetime(sofa_df_merged['ICU_admit']).dt.round('1440min')
    
    sofa_df_merged['day'] = sofa_df_merged['day']-1 #putting the epoch days so that 0 = the first day
    
    #approximating the charttime of the time associated with each sofa score. adding on days to icuadmit. 
    sofa_df_merged['approx_charttime']=pd.to_timedelta((sofa_df_merged['day'])*24, unit='h') + pd.to_datetime(sofa_df_merged['ICU_admit'])

    #rounding down the charttime to the day, so hours and minutes are ignored (just like t_0)
    sofa_df_merged['floor_charttime'] = sofa_df_merged['approx_charttime'].apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, 24*(dt.hour//24))) 
    
    sofa_df_merged['floor_time_var'] = pd.to_datetime(sofa_df_merged[time_var]).apply(lambda dt: datetime.datetime(dt.year, dt.month, dt.day, 24*(dt.hour//24))) 
    
    sofa_df_72= sofa_df_merged.loc[
        (sofa_df_merged['floor_charttime'].between(
            (pd.to_datetime(sofa_df_merged['floor_time_var'])+ pd.to_timedelta(lower_window, unit='d')),
            (pd.to_datetime(sofa_df_merged['floor_time_var'])+ pd.to_timedelta(upper_window, unit='d')+ pd.to_timedelta(1, unit='h')) #added 1hr timebuffer incase between is set as less than greater than
        )),:]
    
    return(sofa_df_72.drop(['floor_time_var','floor_charttime'], axis=1))


time: 35.4 ms


In [209]:
sofa_df_72= sofa_day_window_filter(sofa_pan_df, time_var= time_var, lower_window= lower_window, upper_window=upper_window)

time: 856 ms


In [210]:
sofa_df_72= sofa_day_window_filter(sofa_pan_df, time_var= time_var, lower_window= lower_window, upper_window=upper_window)
filter_subset_save(sofa_df_72, savename="sofa", save=True, return_df=False, time_filter_override=True)
del(sofa_df_72)

time: 697 ms


# saving as csv
## everything below is Depreciated as of 5/16/19: 
haven't need to save the data pretimewindow filtering for a year. this isn't being used anywhere and is just wasting space. 

In [ ]:
# #####NEED TO CLEAN UP PATHWAY CODING

# pd.DataFrame(vaso_dose_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vaso_dose_df.csv' %(date))
# pd.DataFrame(ventsettings_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventsettings_df.csv' %(date))
# # pd.DataFrame(ventcount_df).to_csv(
# #     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcount_df.csv' %(date)) #not useful

# # pd.DataFrame(echodata_df).to_csv(
# #     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_echodata_df.csv' %(date))
# pd.DataFrame(weightfirstday_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_weightfirstday_df.csv' %(date))
# pd.DataFrame(heightfirstday_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_heightfirstday_df.csv' %(date))
# pd.DataFrame(labs_all_nosummary_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_labs_all_nosummary_df.csv' %(date))

# pd.DataFrame(vitals_all_nosummary_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_df.csv' %(date))

# # pd.DataFrame(gcsall_df).to_csv(
# #     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcsall_df.csv' %(date))

# pd.DataFrame(urine_output_all_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_urine_output_all_df.csv' %(date))
# pd.DataFrame(uti_all_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_uti_all_df.csv' %(date))
# pd.DataFrame(bg_all_nosummary_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_bg_all_nosummary_df.csv' %(date))

# #timewindowed
# pd.DataFrame(ventcategory_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcategory_df.csv' %(date))
# pd.DataFrame(rrt_merged_allpt_df).to_csv( 
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_rrt_all_df.csv' %(date)) #timewindowed
# pd.DataFrame(gcs72_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcs72_df.csv' %(date)) #gcs within time window here
# pd.DataFrame(sofa_df_72).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_sofa_df_72.csv' %(date))

# time window filtering 8/28/18  (updated 1/28/19)
saving a new version of each clincal variable dataframe that is filtered to only 72 hour window after t_0 for each icustay_id 

- need to organize this better 

### reading in data if needed

In [ ]:
# #final_pt_df2 = pd.read_csv('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/16082018_final_pt_df2.csv', index_col=0)

# #large import of all data 

# date= '27082018'

# vaso_dose_df =pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vaso_dose_df.csv' %(date), index_col=0)
# ventsettings_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventsettings_df.csv' %(date), index_col=0)
# ventcategory_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcategory_df.csv' %(date), index_col=0)
# ventcount_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_ventcount_df.csv' %(date), index_col=0)

# echodata_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_echodata_df.csv' %(date), index_col=0)
# weightfirstday_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_weightfirstday_df.csv' %(date), index_col=0)
# heightfirstday_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_heightfirstday_df.csv' %(date), index_col=0)
# labs_all_nosummary_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_labs_all_nosummary_df.csv' %(date), index_col=0)
# vitals_all_nosummary_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_df.csv' %(date), index_col=0)
# gcsall_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_gcsall_df.csv' %(date), index_col=0)
# rrt_all_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_rrt_all_df.csv' %(date), index_col=0)
# urine_output_all_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_urine_output_all_df.csv' %(date), index_col=0)
# uti_all_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_uti_all_df.csv' %(date), index_col=0)
# bg_all_nosummary_df=pd.read_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_bg_all_nosummary_df.csv' %(date), index_col=0)


In [ ]:
# def time_window_filter(df, final_pt_df2,timecol,time_var='t_0', lower_window=0, upper_window=3):
#     try:
#         df= pd.merge(df, final_pt_df2[['icustay_id',time_var]], left_on= 'icustay_id', right_on = 'icustay_id') #n=240317
#         df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
#         df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
#         df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:]
#     except KeyError or NameError:
#         df= pd.merge(df, final_pt_df2[['hadm_id',time_var]], left_on= 'hadm_id', right_on = 'hadm_id') #n=240317
#         df['delta']= pd.to_datetime(df[timecol]) - pd.to_datetime(df[time_var])
#         df_after_t0= df.loc[df.loc[:,'delta']>= pd.Timedelta(days=lower_window),:]
#         df_after_t0= df_after_t0.loc[df_after_t0.loc[:,'delta']<= pd.Timedelta(days=upper_window),:]
#     return(df_after_t0)

In [ ]:
# ###list of data w/ 72 hour adjustments
# #vaso_dose_df - vaso_dose_72 -
# #ventsettings_df - ventsettings_72 -
# #ventcategory_df - ventcategory_df - 
# #echodata_df - echodata_72 -
# #labs_all_nosummary_df- labs_all_nosummary_72 -
# #vitals_all_nosummary_df - vitals_all_nosummary_72 -
# #gcsall_df - Gcs72_df **modified the gcs in python above. within 72 hour window. -
# #rrt_all_df - rrt_merged_allpt_df *modified the rrt in python above. within 72 hour window.  -
# #uti_all_df - uti_all_72 -
# #bg_all_nosummary_df - bg_all_nosummary_72
# #sofa_df_72 #already within 72hour window

# #urine_output_all_df - . xxx this also doesn't have times, but this won't be used much so i didn't bother editing.

# vaso_dose_72= time_window_filter(vaso_dose_df, final_pt_df2, 'starttime',time_var=time_var, lower_window=lower_window, upper_window=upper_window)
# ventsettings_72= time_window_filter(ventsettings_df, final_pt_df2, "charttime",time_var=time_var, lower_window=lower_window, upper_window=upper_window)
# # echodata_72= time_window_filter(echodata_df, final_pt_df2, 'charttime',time_var=time_var, lower_window=lower_window, upper_window=upper_window)
# labs_all_nosummary_72= time_window_filter(labs_all_nosummary_df, final_pt_df2, "charttime",time_var=time_var, lower_window=lower_window, upper_window=upper_window)
# vitals_all_nosummary_72 = time_window_filter(vitals_all_nosummary_df, final_pt_df2, "charttime",time_var=time_var, lower_window=lower_window, upper_window=upper_window)
# uti_all_72 = time_window_filter(uti_all_df, final_pt_df2, 'charttime',time_var=time_var, lower_window=lower_window, upper_window=upper_window)
# bg_all_nosummary_72 = time_window_filter(bg_all_nosummary_df, final_pt_df2, 'charttime',time_var=time_var, lower_window=lower_window, upper_window=upper_window)

# filtering to patients with bare minimum vital numbers
06.1-QC_and_missingness found that 3% or so of patients don't have baseline vitals counts. this is filtering the patients to only those who have this baseline value.

as of 10/12/18, this code has not been implemented in here and is performed after importing. (updated below to be self contained in 1/28/19)

In [ ]:
# date= '25012019'

# vitals_all_nosummary_df =pd.read_csv(
#      Path('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_df.csv' %(date)), index_col=0)


In [ ]:
# #using origional criteria to find pts who have atleast 1 spo2 reading within 3 days of t_0

# #The idea is that this should be the bare minimum amount of data for a patient, and without it, it's likely the physicians did not suspect an infection in these patients.

# vitals_filter = time_window_filter(vitals_all_nosummary_df, final_pt_df2, "charttime",time_var='t_0', lower_window=-1, upper_window=3)

# vitals_filter= vitals_filter.loc[
#     vitals_filter['vitalid'].notnull(),:]#.count() #6930 NULL values

# icustay_id_vitals = (vitals_filter.loc[
#     vitals_filter.loc[:,'vitalid']=='SpO2','icustay_id'
#         ].unique())

# len(icustay_id) #8731
# len(icustay_id_vitals) #8629
# subject_id_vitals=list(final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),'subject_id'])
# hadm_id_vitals= list(final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),'hadm_id'])
# icustay_id_vitals= list(final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),'icustay_id'])
# del(vitals_filter)

In [ ]:
# vitals_all_nosummary_72= df_subset(vitals_all_nosummary_72)
# pd.DataFrame(vitals_all_nosummary_72).to_csv("/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_vitals_all_nosummary_72.csv" %date) #final cohort database n=11493 subject_id’s (7/6/18)


In [ ]:
# #saving this slightly reduced cohort for those who have sufficient vitals
# # date= '25012019'
# final_pt_df2_v=final_pt_df2.loc[final_pt_df2.loc[:,'icustay_id'].isin(icustay_id_vitals),:]
# pd.DataFrame(final_pt_df2_v).to_csv("/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s_final_pt_df2_v.csv" %date) #final cohort database n=11493 subject_id’s (7/6/18)


In [ ]:
# def df_subset(df):
#     try:
#         df = df.loc[df.loc[:,'icustay_id'].isin(icustay_id_vitals),:]
#     except KeyError or NameError:
#         try:
#             df = df.loc[df.loc[:,'hadm_id'].isin(hadm_id_vitals),:]
#         except KeyError or NameError:
#             df = df.loc[df.loc[:,'subject_id'].isin(subject_id_vitals),:]
#     return(df)

In [ ]:
# # subsetting each dataframe to only patients in final_patients_df2_v:
# dataframe_list= [
#     age_df, ventcategory_df, vaso_dose_72, #echodata_72,
#     labs_all_nosummary_72, weightfirstday_df,
#     heightfirstday_df, vitals_all_nosummary_72,
#     uti_all_72, bg_all_nosummary_72,
#     rrt_merged_allpt_df, gcs72_df, sofa_df_72
# ]

# (
# age_df, ventcategory_df, vaso_dose_72, #echodata_72,
# labs_all_nosummary_72, weightfirstday_df,
# heightfirstday_df, vitals_all_nosummary_72,
# uti_all_72, bg_all_nosummary_72,
# rrt_merged_allpt_df, gcs72_df, sofa_df_72
# ) = (df_subset(df) for df in dataframe_list)

In [ ]:
# bg_all_nosummary_72

In [ ]:
# #date= '27082018'
# # date= '25012019'

# folder="48_hr_window"#"72_hr_window"

# pd.DataFrame(age_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_pt_info_df.csv' %(folder,date))
# pd.DataFrame(vaso_dose_72).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_vaso_dose_72.csv' %(folder,date))

# pd.DataFrame(vaso_dose_72).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_vaso_dose_72.csv' %(folder,date))
# pd.DataFrame(ventsettings_72).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_ventsettings_72.csv' %(folder,date))

# #vent category and count are already 72hour
# pd.DataFrame(ventsettings_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_ventsettings_df.csv' %(folder,date))
# pd.DataFrame(ventcategory_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_ventcategory_df.csv' %(folder,date))

# # pd.DataFrame(echodata_72).to_csv(
# #     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/72_hr_window/%s_echodata_72.csv' %(date))
# pd.DataFrame(labs_all_nosummary_72).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_labs_all_nosummary_72.csv' %(folder,date))

# #vent category and count are already limited to first day
# pd.DataFrame(weightfirstday_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_weightfirstday_df.csv' %(folder,date))
# pd.DataFrame(heightfirstday_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_heightfirstday_df.csv' %(folder,date))

# pd.DataFrame(vitals_all_nosummary_72).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_vitals_all_nosummary_72.csv' %(folder,date))
# pd.DataFrame(uti_all_72).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_uti_all_72.csv' %(folder,date))
# pd.DataFrame(bg_all_nosummary_72).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_bg_all_nosummary_72.csv' %(folder,date))

# pd.DataFrame(rrt_merged_allpt_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_rrt_merged_allpt_df.csv' %(folder,date))
# pd.DataFrame(gcs72_df).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_gcs72_df.csv' %(folder,date))

# pd.DataFrame(sofa_df_72).to_csv(
#     '/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/raw/csv/%s/%s_sofa_df_72.csv' %(folder,date))
